## 交叉验证

In [2]:
import numpy as np
from sklearn import datasets

In [3]:
# 使用 手写识别数据
digits = datasets.load_digits()
X = digits.data
y = digits.target

### 使用 train test split 模式, 手动搜索

In [6]:
# 先用原先一直用的 train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666, test_size=0.4)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k, best_p, best_score = k, p, score

print("best k =", best_k)
print("best p =", best_p)
print("best score =", best_score)

best k = 3
best p = 4
best score = 0.9860917941585535


### 使用交叉验证

In [13]:
# sklearn 中的 交叉验证用法
from sklearn.model_selection import cross_val_score
knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train, cv=3)

array([0.98895028, 0.97777778, 0.96629213])

In [11]:
# 使用交叉验证来搜索, 注意 fit 和 score 都集成到了 cv 的过程， 而且整个过程没有用到 X_test 和 y_test
best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        score = np.mean(cross_val_score(knn_clf, X_train, y_train, cv=3))
        if score > best_score:
            best_k, best_p, best_score = k, p, score

print("best k =", best_k)
print("best p =", best_p)
print("best score =", best_score)

best k = 2
best p = 2
best score = 0.9823599874006478


In [14]:
# 用的到的模型参数来测试

In [15]:
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.980528511821975

### 回顾网格搜索

In [16]:
# 其实 sklearn 中的 网格搜索就使用了交叉验证的方法，GrideSearchCV 后面两个字母就代表 Cross Validation

In [26]:
from sklearn.model_selection import GridSearchCV

knn_clf = KNeighborsClassifier()

param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)],
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:    3.5s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [19]:
# 注意这个输出：Fitting 3 folds for each of 45 candidates, totalling 135 fits
# 表示： 采用3份交叉验证，每份验证要做45（9*5）次拟合， 共 135 (45*3) 次拟合

In [20]:
grid_search.best_score_

0.9823747680890538

In [21]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [22]:
# 再用网格搜索出来的模型测试
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.980528511821975

In [23]:
# 和我们自己手动用交叉验证循环查找的结果一致。

### CV 参数

In [24]:
# sklearn 中的 CV 默认分为 3 份， 可以调整

In [25]:
cross_val_score(knn_clf, X_train, y_train, cv=5)

array([0.99543379, 0.97716895, 0.97685185, 0.98130841, 0.97142857])

In [ ]:
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=1)